# Problem Sheet 10 - Tucker decomposition

**Submission until January 30 at 7 p.m. in the corresponding folder in StudIP.**

This exercise sheet is about the Tucker decomposition. The Tucker decomposition for a tensor $X \in \mathbb{R}^{n_{1} \times \dots \times n_{d}}$ is denoted by $[G; U_{1}, \dots, U_{d} ]$ such that

\begin{align*}
    [G; U_{1}, \dots, U_{d} ] :=  \ G \times_{1} U_{1} \times_{2} U_{2} \times_{3} \dots \times_{d} U_{d} = X.
\end{align*}

Unfortunately, the Tucker decomposition is not unique. Therefore we will consider the Higher-order singular value decomposition (HOSVD) to compute one Tucker decomposition of a tensor. The HoSVD will be repeated later.

# Task 1: Load Miranda Turbulent Flow tensor

The first task is just about loading special packages und downloading a certain data set. We will download the **Miranda Turbulent Flow** tensor `density.mat` from the GitLab repository https://gitlab.com/tensors/tensor_data_miranda_sim. There you can find some more information about the data set. We will use this tensor in the fourth task to see a nice application for tensor decomposition.

Execute the following code to load import packages.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.utils.extmath import randomized_svd
import scipy.io


Execute the following code to install the package `wget` and download the **Miranda Turbulent Flow** tensor `density.mat`.

In [2]:
import sys
!{sys.executable} -m pip install wget
import wget
url = "https://gitlab.com/tensors/tensor_data_miranda_sim/-/raw/main/density.mat?ref_type=heads&inline=false"
response = wget.download(url, "density.mat")

Execute the following code to load the **Miranda Turbulent Flow** tensor and save a subtensor of it into `data`.

In [3]:
file = scipy.io.loadmat('density.mat')
data = file['density'][500:900,:,:]

### ATTENTION! 
Please keep in mind that we want to consider a tensor of size $400 \times 256 \times 256$. To work with such a tensor can be numerically very expensive. Therefore please avoid unnecessary loops or unnecessary calculations of the SVD. If you have to compute the SVD use the function `randomized_svd`. It takes a matrix and a rank as input. For further information look at https://scikit-learn.org/stable/modules/generated/sklearn.utils.extmath.randomized_svd.html. 

# Task 2: Higher-order singular value decomposition (HOSVD)

In this task we want to consider the Higher-order singular value decompostion (HOSVD). Let's repeat the HOSVD. 

The HOSVD with multirank $r = (r_{1}, \dots, r_{d})$ for a tensor $X \in \mathbb{R}^{n_{1} \times \dots \times n_{d}}$ is given by the set $[G; U_{r_{1}}, \dots, U_{r_{d}} ]$ and it is determined by the truncated SVD of the unfoldings $X_{(k)}$ of $X$. The truncated SVD of the unfoldings $X_{(k)}$ is given as follows

\begin{align*}
    X_{(k)} \approx U_{r_{k}} \Sigma_{r_{k}} V^{\mathrm{T}}_{r_{k}}, \qquad \forall k=1,\dots,d,
\end{align*}
where $r_{k} \leq \mathrm{rank}(X_{(k)})$, $U_{k} \in \mathbb{R}^{n_{k} \times r_{k}}$, $\Sigma_{k} = \mathrm{diag}(\sigma_{1}, \dots, \sigma_{r_{k}}) \in \mathbb{R}^{r_{k} \times r_{k}}$ and $V_{k} \in \mathbb{R}^{ (n_{1} \cdot \ \dots \ \cdot n_{k-1} \cdot n_{k+1} \cdot \ \dots \ \cdot n_{d}) \times r_{k}}$. \
Then the core $G$ is given by

\begin{align*}
    G = X \times_{1} U^{\mathrm{T}}_{r_{1}} \times_{2} U^{\mathrm{T}}_{r_{2}} \times_{3} \dots \times_{d} U^{\mathrm{T}}_{r_{d}} .
\end{align*}

Now the HOSVD of $X$ with multirank $r$ is given by

\begin{align*}
    [G; U_{r_{1}}, \dots, U_{r_{d}} ] :=& \  G \times_{1} U_{r_{1}} \times_{2} U_{r_{2}} \times_{3} \dots \times_{d} U_{r_{d}} \\
    =& \ X \times_{1} U_{r_{1}} U^{\mathrm{T}}_{r_{1}} \times_{2} U_{r_{2}} U^{\mathrm{T}}_{r_{2}} \times_{3} \dots \times_{d} U_{r_{d}} U^{\mathrm{T}}_{r_{d}} \approx X.
\end{align*}

**Task: Implement a function which computes the HOSVD. The function should return the matrices $G,U_{1},\dots, U_{d}$ for given tensor $X$ and given multirank $r$. (3 points)**

Hint: To compute the HOSVD, follow the defintion above. Compute the 1st unfolding of X and then the corresponding truncated SVD. In the next step compute the 2nd unfolding of X and the corresponding truncated SVD. Repeat this until there is no more unfolding left.

In [4]:
ex = np.arange(start=1, stop=25).reshape(3,4, 2, order="F")

In [5]:
def get_unfolding(tensor, mode):
    if(mode<1):
        raise ValueError('Bad range for mod')
    return np.moveaxis(tensor, mode-1, 0).reshape(tensor.shape[mode-1], np.prod(tensor.shape)//tensor.shape[mode-1], order = 'F')

In [6]:
def modek_prod(tensor, matrix, mode):
    return np.swapaxes(np.swapaxes(tensor, mode - 1, -1).dot(matrix.T), mode - 1, -1)

In [7]:
def hosvd_decomposition(tensor, multirank):
    u_list = [] #list of the u matrix 
    for i in range(1, len(tensor)+1):
        unfold = get_unfolding(tensor, i)
        U, s, Vh = randomized_svd(unfold, n_components=multirank[i-1], random_state=0) #truncated SVD
        u_list.append(U) 
        
    #compute G
    G = modek_prod(tensor, u_list[0].T, 1)
    for i in range(1, len(multirank)):
        G = modek_prod(G, u_list[i].T, i+1)
        
    #compute the approx X
    X_approx = modek_prod(G, u_list[0], 1)
    for i in range(1, len(multirank)):
        X_approx = modek_prod(X_approx, u_list[i], i+1)
    return X_approx, G, u_list
          

In [8]:
X_approx, G, u_list = hosvd_decomposition(ex, (3, 4, 2))
G.shape

(3, 4, 2)

**Task: Apply your function to the given tensor $X$. Try different values for the multirank $r$ and compute the distances between the HOSVD and $X$. (2 points)**

In [28]:
X = np.arange(1,61).reshape((3, 4, 5), order = "F")
#Different multirank examples
X_app, G, u_list = hosvd_decomposition(X, (3, 2, 2))
print(f'The rank of X_aproxx {(3, 2, 2)} and the original {X.shape}')
print(f'The Distance between the HOSVD and the original tensor is {np.sqrt(np.sum((X-X_app)**2))}')
print('----')
X_app, G, u_list = hosvd_decomposition(X, (3, 4, 4))
print(f'The rank of X_aproxx {(3, 4, 4)} and the original {X.shape}')
print(f'The Distance between the HOSVD and the original tensor is {np.sqrt(np.sum((X-X_app)**2))}')

The rank of X_aproxx (3, 2, 2) and the original (3, 4, 5)
The Distance between the HOSVD and the original tensor is 1.228875971116375e-13
----
The rank of X_aproxx (3, 4, 4) and the original (3, 4, 5)
The Distance between the HOSVD and the original tensor is 9.648189413995894e-14


**Task: If you have a tensor $X \in \mathbb{R}^{5 \times 4 \times 8}$ and you have the HOSVD of multirank $r = (5,4,7)$ denoted by $T_{r}$, what is the error between $X$ and $T_{r}$ you would expect? (1 points)**

I won't except a big error since for the first 2 dimensions of the multirank stay basically the same for the approximation and for the last dimension we only go a dimension lower with the compression of the rank.

# Task 3: Approximation of tensors

In this task we want to approximate a tensor by the HOSVD with smaller multirank than the original tensor. We have learned two different approaches to indicate a good approximation, the compression rate and the approximation error. For a tensor $X \in \mathbb{R}^{n_{1} \times \dots \times n_{d}}$ and multirank $r=(r_{1}, \dots, r_{d})$ we denote $T_{r}$ as the HoSVD with multirank $r$ of $X$. \
Then the approximation error is defined by

\begin{align*}
    \mathrm{approxerr} := \frac{\Vert X - T_{r} \Vert}{\Vert X \Vert},
\end{align*}

and the compression ratio is defined by

\begin{align*}
    \mathrm{comprat} := \frac{\prod\limits_{k=1}^{d} r_{k} + \sum\limits_{k=1}^{d} r_{k} n_{k}}{\prod\limits_{k=1}^{d} n_{k}}.
\end{align*}
A goal for the approximation of a tensor could be the minimization of the approximation error or the compression ratio.

In the following we are interested in finding $T_{r}$ which minimizes the compression ratio under the condition that the approximation error is less than a certain accuracy $\epsilon$. So we want to solve

\begin{align*}
    \underset{r = (r_{1},\dots,r_{d})}{\mathrm{min}} & \ \frac{\prod\limits_{k=1}^{d} r_{k} + \sum\limits_{k=1}^{d} r_{k} n_{k}}{\prod\limits_{k=1}^{d} n_{k}} \qquad \mathrm{s.t.} \ \frac{\Vert X - T_{r} \Vert}{\Vert X \Vert} \leq \epsilon
\end{align*}

To obtain a good approximation of the solution one can use the fact that

\begin{align*}
    \Vert X - T_{r} \Vert^{2} \leq \sum\limits_{i_{1} = r_{1} + 1}^{n_{1}} \left( \sigma_{i_{1}}^{(1)} \right)^{2} + \sum\limits_{i_{2} = r_{2} + 1}^{n_{2}} \left( \sigma_{i_{2}}^{(2)} \right)^{2} + \dots + \sum\limits_{i_{d} = r_{d} + 1}^{n_{d}} \left( \sigma_{i_{d}}^{(d)} \right)^{2}, 
\end{align*}
where $\sigma_{j}^{(k)}$ is the $j$-th singular value of the k-th unfolding $X_{(k)}$. \
To obtain the approximation of the minimization problem one can maximize the sum of the singular values w.r.t. $r = (r_{1}, \dots, r_{k})$ such that the sum is still smaller than $\epsilon^{2}$. So if we take the smallest $r = (r_{1}, \dots, r_{k})$ such that

\begin{align*}
    \sum\limits_{i_{1} = r_{1} + 1}^{n_{1}} \left( \sigma_{i_{1}}^{(1)} \right)^{2} + \sum\limits_{i_{2} = r_{2} + 1}^{n_{2}} \left( \sigma_{i_{2}}^{(2)} \right)^{2} + \dots + \sum\limits_{i_{d} = r_{d} + 1}^{n_{d}} \left( \sigma_{i_{d}}^{(d)} \right)^{2} \leq \epsilon^{2} \Vert X \Vert^2
\end{align*}

holds, then we have found a good approximation of the solution for the minimization problem. \
To find this approximation $r = (r_{1}, \dots, r_{k})$ you have to compute all singular values for each unfolding $X_{(k)}$. Then sum up all singular values starting with smallest one in ascending order until you reach $\epsilon^{2}$. Now you know which singular values and how many singular you have to drop to reach the approximation.

**Task: Write a function which computes the best multirank $r$ for the above defined minimization problem. The function should have a tensor $X$ and an array containg different values for the accuracy $\epsilon$ as inputs. The function should return the multirank $r$ for each accuracy $\epsilon$ in your array. (4 points)**

In [12]:
np.concatenate((np.full(10, 3),np.array([1, 2, 3])))

array([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 2, 3])

In [13]:
def custom_sort_tupel(arr1, arr2):
    combined = list(zip(arr1, arr2)) #zip the list in an array of tuples
    combined.sort(key=lambda x: x[0]) #sort only by the first argument in the tuples
    arr1[:], arr2[:] = zip(*combined) #unzip in two arrays, where the first one is sorted and in the second one the elements followed the interchance of pos int the first one
    return arr1, arr2

In [29]:
def minim_rank(X, acc_arr):
    sing_values = []
    rank = []
    x_shape = list(X.shape)
    minim_rank = []

    #singular values and their according dimension/mode where they were calculated
    for i in range(1, len(X)+1):
        unfold = get_unfolding(X, i)
        sing_values = sing_values + list(np.linalg.svd(unfold)[1])
        rank = rank + list(np.full(len(np.linalg.svd(unfold)[1]), i))
    sing_values, rank = custom_sort_tupel(sing_values, rank) #sort the singular values and also intechange the rank pos in the rank array 

    # iterate through the accuracies
    for acc in acc_arr:
        sum_eig_val = 0
        index_stop = len(sing_values)
        min_r=list(X.shape)
        #sum the singular values
        for value in sing_values:
            sum_eig_val += value
            if(sum_eig_val>acc**2): #braeak condition
                index_stop = sing_values.index(value)
                break
        for i in range(index_stop): #substract from the actual rank how much we can reduce our dimensions
            min_r[rank[i]-1] -= 1
        minim_rank.append(tuple(min_r))
        
    return minim_rank

**Task: Compute the multirank $r$ of the given tensor $X$ for the accuracies $0.01$, $0.02$ and $0.1$. Compute the remaining HOSVDs $T_{r}$ and print the approximation error. (2 points)**

In [30]:
X = np.arange(1,61).reshape((3, 4, 5), order = "F")
print(X.shape)
minim_rank(X, [0.01, 0.02, 0.1])

(3, 4, 5)


[(2, 2, 2), (2, 2, 2), (2, 2, 2)]

In [51]:
X = np.arange(1,61).reshape((3, 4, 5), order = "F")
X_app, G, u_list = hosvd_decomposition(X, (2, 2, 2))
eroor = get_unfolding((X-X_app), 2)
print(np.linalg.norm(eroor, 'fro'))

1.0566057328026412e-13


# Task 4: Application 

In the last part we want to extract some information about the **Miranda Turbulent Flow** tensor. To do this we need our above defined functions.

In the first step we want approximate the **Miranda Turbulent Flow** tensor. We want to compute HOSVD of the tensor under given accuracy $\epsilon$ and we want to look at the corresponding approximation error, core size and compression rate.

**Task: Write a function which computes the HOSVD of a tensor for given accuracies. The function should return a dataframe which contains the accuracies, approximation error, core size and compression rate. The function should also return the corresponding HOSVDs. (4 points)**

Hint: You can use `pandas.DataFrame` to construct such dataframe. Your resulting dataframe should look as follows.

Accuracy | Approx. error | Core size | Compression ratio
---|---|--- | ---
0.0001 | 0.000099 | 240x163x159 | 0.244084
0.001  | 0.000982 | 160x109x107 | 0.075736
0.01   | 0.009465 |  75x52x52   | 0.009896
0.1    | 0.077167 |  10x9x8     | 0.000346

**Task: Apply your function to the Miranda Turbulent Flow tensor `data` and plot the dataframe. (1 points)**

Hint: Keep in mind that the Miranda Turbulent Flow tensor is large. So this process can take 10-20 minutes.

The following function creates a $3 \times 4$ subplot. Each row corresponds to a certain slice of the tensor and each column contains the approxmations of the slices w.r.t. to a certain accuracy. The function has 4 inputs. The first input is the original which we defined as `data`. The second input is the HOSVD w.r.t. accuracy 0.001, the third input is the HOSVD w.r.t. accuracy 0.01 and the fourth input is the HOSVD w.r.t. accuracy 0.1.

Execute the following code.

In [18]:
def plot_slice(data,tensor1,tensor2,tensor3):

    fig, axs = plt.subplots(3, 4, figsize=(15, 15))
    axs[0, 0].imshow(data[:,64,:])
    axs[0, 0].set_ylabel('Data[:,64,:]')
    axs[0, 0].set_title('original')
    axs[0, 0].set_xticks([])
    axs[0, 0].set_yticks([])

    axs[0, 1].imshow(tensor1[:,64,:])
    axs[0, 1].set_title('tol = 0.001')
    axs[0, 1].axis("off")

    axs[0, 2].imshow(tensor2[:,64,:])
    axs[0, 2].set_title('tol = 0.01')
    axs[0, 2].axis("off")

    axs[0, 3].imshow(tensor3[:,64,:])
    axs[0, 3].set_title('tol = 0.1')
    axs[0, 3].axis("off")

    axs[1, 0].imshow(data[:,128,:])
    axs[1, 0].set_ylabel('Data[:,128,:]')
    axs[1, 0].set_xticks([])
    axs[1, 0].set_yticks([])

    axs[1, 1].imshow(tensor1[:,128,:])
    axs[1, 1].axis("off")

    axs[1, 2].imshow(tensor2[:,128,:])
    axs[1, 2].axis("off")

    axs[1, 3].imshow(tensor3[:,128,:])
    axs[1, 3].axis("off")

    axs[2, 0].imshow(data[:,192,:])
    axs[2, 0].set_ylabel('Data[:,192,:]')
    axs[2, 0].set_xticks([])
    axs[2, 0].set_yticks([])

    axs[2, 1].imshow(tensor1[:,192,:])
    axs[2, 1].axis("off")

    axs[2, 2].imshow(tensor2[:,192,:])
    axs[2, 2].axis("off")

    axs[2, 3].imshow(tensor3[:,192,:])
    axs[2, 3].axis("off")

    plt.show()

**Task: Use the function to plot your computed HOSVDs. (1 points)**

**Task: Describe what you can see in the plots. Explain the difference between the images in one row. (2 points)**